In [5]:
def generate_gpt3_prompt(answers):
    allowed_fruits = ["oranges", "apples", "pears", "grapes", "watermelon", "lemon", "lime"]

    # Apply rules based on answers
    if answers['go_out_to_party'] == 'yes':
        allowed_fruits = list(set(allowed_fruits) & set(["apples", "pears", "grapes", "watermelon"]))

    if answers['flavours'] == 'cider':
        allowed_fruits = list(set(allowed_fruits) & set(["apples", "oranges", "lemon", "lime"]))

    if answers['flavours'] == 'sweet':
        allowed_fruits = list(set(allowed_fruits) & set(["watermelon", "oranges"]))

    if answers['flavours'] == 'waterlike':
        allowed_fruits = list(set(allowed_fruits) & set(["watermelon"]))

    if 'grapes' in answers.get('flavours', []):
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit != 'watermelon']

    if answers['texture_not_like'] == 'smooth':
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit != 'pears']

    if answers['texture_not_like'] == 'slimy':
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit not in ["watermelon", "lime", "grapes"]]

    if answers['texture_not_like'] == 'waterlike':
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit != 'watermelon']

    if answers['price_range'] == '< $3':
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit not in ["lime", "watermelon"]]

    if answers['price_range'] in ['> $4 and < $7', '> $4 and < $7']:
        allowed_fruits = [fruit for fruit in allowed_fruits if fruit not in ["pears", "apples"]]

    # Generate GPT-3 prompt
    prompt = f"Based on your answers:\n" \
             f"- Go out to party: {answers['go_out_to_party']}\n" \
             f"- Flavours: {answers['flavours']}\n" \
             f"- Texture not liked: {answers['texture_not_like']}\n" \
             f"- Price range: {answers['price_range']}\n" \
             f"The recommended fruits are: {', '.join(allowed_fruits)}."

    return prompt

In [6]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/get_recommendation', methods=['POST'])
def get_recommendation():
    try:
        data = request.get_json()
        if data:
            answers = {
                'go_out_to_party': data.get('go_out_to_party', ''),
                'flavours': data.get('flavours', ''),
                'texture_not_like': data.get('texture_not_like', ''),
                'price_range': data.get('price_range', '')
            }

            # Call the function to generate GPT-3 prompt
            prompt = generate_gpt3_prompt(answers)

            return jsonify({'prompt': prompt})
        else:
            return jsonify({'error': 'Invalid JSON format'}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
##for api testing

curl -X POST -H "Content-Type: application/json" -d '{
  "go_out_to_party": "no",
  "flavours": "sweet",
  "texture_not_like": "smooth",
  "price_range": "$5"
}' http://localhost:5000/get_recommendation
